# Analyze data from education systems

In [ ]:
from math import prod
import pandas as pd
import matplotlib .pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## Methods cell

In [ ]:
def df_initial_analysis(df, name_df):
    """
    Initial analysis on the DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame to analyze.
        name_df (str): DataFrame name.

    Returns:
        None.
        Print the initial analysis on the DataFrame. 
    """
    if df.empty:
        print("The", name_df, "dataset is empty. Please verify the file.")
    else:
        empty_cols = [col for col in df.columns if df[col].isna().all()] # identifying empty columns
        df_rows_duplicates = df[df.duplicated()] #identifying full duplicates rows
        
        # Creating a dataset based on Type object and records by columns
        type_cols = df.dtypes.apply(lambda x: x.name).to_dict() 
        df_resume = pd.DataFrame(list(type_cols.items()), columns = ["Name", "Type"])
        df_resume["Records"] = list(df.count())
        
        print("\nInitial Analysis of", name_df, "dataset")
        print("--------------------------------------------------------------------------")
        print("- Dataset shape:                 ", df.shape[0], "rows and", df.shape[1], "columns")
        print("- Total of NaN values:           ", df.isna().sum().sum())
        print("- Percentage of NaN:             ", round((df.isna().sum().sum() / prod(df.shape)) * 100, 2), "%")
        print("- Total of full duplicates rows: ", df_rows_duplicates.shape[0])
        print("- Total of empty columns:        ", len(empty_cols))
        print(" + The empty column is:          ", empty_cols) if len(empty_cols) == 1 \
            else (print(" + The empty column are:          ", empty_cols) if len(empty_cols) >= 1 else None)
        print("\n- Type object and records by columns")
        print("--------------------------------------------------------------------------")
        print(df_resume.sort_values("Records", ascending=False))


def comparison_dfs(df_original, df_copy):
    """
    Create a DataFrame based on comparación between the original DataFrame and the copy DataFrame.
    Args:
        df_original (pandas.DataFrame): DataFrame original.
        df_copy (pandas.DataFrame): DataFrame copy.
    Returns:
        df_comparison (pandas.DataFrame): DataFrame comparison.
    """
    df_comparison = pd.DataFrame({
        "Moment" : ["Before", "After"],
        "Total values" : [prod(df_original.shape), prod(df_copy.shape)], 
        "Valid values" : [(prod(df_original.shape)-df_original.isna().sum().sum()), (prod(df_copy.shape)-df_copy.isna().sum().sum())],
        "Missing values" : [df_original.isna().sum().sum(), df_copy.isna().sum().sum()]
    })
    return df_comparison


def remove_columns_by_percentage_of_nan_values(df):
    """
    Remove columns in DataFrame based on percentage of NaN values in columns.
    Args:
        df (pandas.DataFrame): DataFrame to work.
    Returns:
        df (pandas.DataFrame): DataFrame worked.
    """
    percentage = 50 # Removing columns with more than 50% NaN values 

    for col in df.columns:
        if round(((df.shape[0] - df[col].count()) / df.shape[0]) * 100, 2) > percentage:
            print("- The column",  df[col].name, "has been removed.")
            df.drop([col], axis=1, inplace=True)

    return df

## Loading data

In [ ]:
# loading Dataset and removing the empty columns (the empty columns, unnamed, etc.)
country = pd.read_csv("datasets/EdStatsCountry.csv")
serie = pd.read_csv("datasets/EdStatsSeries.csv")
country_serie = pd.read_csv("datasets/EdStatsCountry-Series.csv")
foot_note = pd.read_csv("datasets/EdStatsFootNote.csv")
data = pd.read_csv("datasets/EdStatsData.csv")

## Initial data analysis
- Knowing the entities of the datasets.<br>

    - **Country** dataset **`"EdStatsCountry.csv"`**.<br>
    Des renseignements en général sur l'économie de chaque pays du monde.
    
    - **Serie** dataset **`"EdStatsSeries.csv"`**.<br>
    Des renseignements en peu plus de détails sur les indicateurs économiques.
    
    - **Country-Serie** dataset **`"EdStatsCountry-Series.csv"`**.<br>
    La source des données sur l'information relationnelle entre **`"EdStatsCountry.csv"`** et **`"EdStatsSeries.csv"`**.

    - **Foot Note** dataset **`"EdStatsFootNote.csv"`**.<br>
    L’année d’origine des données et aussi une description sur les indicateurs.

    - **Data** dataset **`"EdStatsData.csv"`**.<br>
    Le dataset principal qui contient en détails le renseignement sur les indicateurs par an.

- Entity relationship diagram.<br>
<br>Il existe une relation entre les datasets à travers les colonnes **`"Country Code"`** et **`"Series Code"`** principalement.
![Entity relationship diagram](img/entity_relationship.png)

### **`"Country"`** dataset

In [ ]:
country.head(3)

In [ ]:
# Describing the numeric columns
country.describe()

In [ ]:
# Resume of dataset
df_initial_analysis(country, "country")

In [ ]:
# Drawing missing values in Country dataset
labels = ["records", "missing values"]
vals = [prod(country.shape)-country.isna().sum().sum(), country.isna().sum().sum()]
explode = (0, 0.01)

pie, ax = plt.subplots(figsize=[10,6])
plt.axis("equal")
plt.pie(x=vals, labels=labels, explode=explode,  autopct="%.1f%%", pctdistance=0.5)
plt.title("Missing values in Country dataset", fontsize=14)
plt.show()

### **`"Serie"`** dataset

In [ ]:
serie.head(3)

In [ ]:
# Describing the numeric columns
serie.describe()

In [ ]:
# Resume of dataset
df_initial_analysis(serie, "serie")

In [ ]:
# Drawing missing values in Serie dataset
labels = ["records", "missing values"]
vals = [prod(serie.shape)-serie.isna().sum().sum(), serie.isna().sum().sum()]
explode = (0, 0.01)

pie, ax = plt.subplots(figsize=[10,6])
plt.axis("equal")
plt.pie(x=vals, labels=labels, explode=explode,  autopct="%.1f%%", pctdistance=0.5)
plt.title("Missing values in Serie dataset", fontsize=14)
plt.show()

### **`"Country-Serie"`** dataset

In [ ]:
country_serie.head(3)

In [ ]:
# Resume of dataset
df_initial_analysis(country_serie, "country_serie")

In [ ]:
# Drawing missing values in Country-Serie dataset
labels = ["records", "missing values"]
vals = [prod(country_serie.shape)-country_serie.isna().sum().sum(), country_serie.isna().sum().sum()]
explode = (0, 0.01)

pie, ax = plt.subplots(figsize=[10,6])
plt.axis("equal")
plt.pie(x=vals, labels=labels, explode=explode,  autopct="%.1f%%", pctdistance=0.5)
plt.title("Missing values in Country-Serie dataset", fontsize=14)
plt.show()

### **`"Foot Note"`** dataset

In [ ]:
foot_note.head(3)

In [ ]:
# Describing the numeric columns
foot_note.describe()

In [ ]:
# Resume of dataset
df_initial_analysis(foot_note, "foot_note")

In [ ]:
# Drawing missing values in Foot Note dataset
labels = ["records", "missing values"]
vals = [prod(foot_note.shape)-foot_note.isna().sum().sum(), foot_note.isna().sum().sum()]
explode = (0, 0.01)

pie, ax = plt.subplots(figsize=[10,6])
plt.axis("equal")
plt.pie(x=vals, labels=labels, explode=explode,  autopct="%.1f%%", pctdistance=0.5)
plt.title("Missing values in Foot-Note dataset", fontsize=14)
plt.show()

In [ ]:
### **`"Data"`** dataset

In [ ]:
data.head(3)

In [ ]:
# Describing the numeric columns
data.describe()

In [ ]:
# Resume of dataset
df_initial_analysis(data, "data")

In [ ]:
# Drawing missing values in Data dataset
labels = ["records", "missing values"]
vals = [prod(data.shape)-data.isna().sum().sum(), data.isna().sum().sum()]
explode = (0, 0.01)

pie, ax = plt.subplots(figsize=[10,6])
plt.axis("equal")
plt.pie(x=vals, labels=labels, explode=explode,  autopct="%.1f%%", pctdistance=0.5)
plt.title("Missing values in Data dataset", fontsize=14)
plt.show()

## Data cleanup
- Creating copies from all datasets.
- Deleting empty columns and empty rows.
- Filtring by list of **`"countries"`** as given in **`"ISO 3166-1"`** and the corresponding **`"ISO 3166-1-alpha-2"`** code elements.<br>
https://datahub.io/core/country-list#resource-data

### Loading data and removing empty columns and empty rows

In [ ]:
country_copy = country.dropna(axis="columns", how="all").dropna(axis="rows", how="all")
serie_copy = serie.dropna(axis="columns", how="all").dropna(axis="rows", how="all")
country_serie_copy = country_serie.dropna(axis="columns", how="all").dropna(axis="rows", how="all")
foot_note_copy = foot_note.dropna(axis="columns", how="all").dropna(axis="rows", how="all")
data_copy = data.dropna(axis="columns", how="all").dropna(axis="rows", how="all")

### Comparing datasets before and after removing empty columns/rows

In [ ]:
# Comparing missing values in Country dataset before and after removing empty columns/rows
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(nrows=5, ncols=2, figsize=[10,20])
labels = ["records", "missing values"]
explode = (0, 0.01)

vals_ax1 = [prod(country.shape)-country.isna().sum().sum(), country.isna().sum().sum()]
ax1.pie(x=vals_ax1, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax1.set_title("Country dataset", fontsize=14)

vals_ax2 = [prod(country_copy.shape)-country_copy.isna().sum().sum(), country_copy.isna().sum().sum()]
ax2.pie(x=vals_ax2, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax2.set_title("Country dataset after removing empty columns/rows", fontsize=14)

vals_ax3 = [prod(serie.shape)-serie.isna().sum().sum(), serie.isna().sum().sum()]
ax3.pie(x=vals_ax3, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax3.set_title("Serie dataset", fontsize=14)

vals_ax4 = [prod(serie_copy.shape)-serie_copy.isna().sum().sum(), serie_copy.isna().sum().sum()]
ax4.pie(x=vals_ax4, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax4.set_title("Serie dataset after removing empty columns/rows", fontsize=14)

vals_ax5 = [prod(country_serie.shape)-country_serie.isna().sum().sum(), country_serie.isna().sum().sum()]
ax5.pie(x=vals_ax5, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax5.set_title("Country-Serie dataset", fontsize=14)

vals_ax6 = [prod(country_serie_copy.shape)-country_serie_copy.isna().sum().sum(), country_serie_copy.isna().sum().sum()]
ax6.pie(x=vals_ax6, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax6.set_title("Country-Serie dataset after removing empty columns/rows", fontsize=14)

vals_ax7 = [prod(foot_note.shape)-foot_note.isna().sum().sum(), foot_note.isna().sum().sum()]
ax7.pie(x=vals_ax7, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax7.set_title("Foot Note dataset", fontsize=14)

vals_ax8 = [prod(foot_note_copy.shape)-foot_note_copy.isna().sum().sum(), foot_note_copy.isna().sum().sum()]
ax8.pie(x=vals_ax8, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax8.set_title("Foot Note dataset after removing empty columns/rows", fontsize=14)

vals_ax9 = [prod(data.shape)-data.isna().sum().sum(), data.isna().sum().sum()]
ax9.pie(x=vals_ax9, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax9.set_title("Data dataset", fontsize=14)

vals_ax10 = [prod(data_copy.shape)-data_copy.isna().sum().sum(), data_copy.isna().sum().sum()]
ax10.pie(x=vals_ax10, labels=labels, autopct="%.1f%%", pctdistance=0.5)
ax10.set_title("Data dataset after removing empty columns/rows", fontsize=14)

plt.tight_layout()
plt.show()

### Comparing **`"Country"`** dataset before and after removing empty columns/rows

In [ ]:
df_comparison = comparison_dfs(country, country_copy)
fig, ax1 = plt.subplots(figsize=(10, 10))
dt = df_comparison.melt(id_vars="Moment").rename(columns=str.title)
splot=sns.barplot(x="Moment", y="Value", data=dt, hue="Variable", ax=ax1)
for p in splot.patches:
    splot.annotate(format(p.get_height(), ".1f"), (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha="center", va="center", xytext=(0, 9), textcoords="offset points")
plt.xlabel("Comparison moments", size=12)
plt.ylabel("Number of records", size=12)
plt.title("Country dataset before and after removing empty columns/rows", size=14)
sns.despine(fig)

### Comparing **`"Serie"`** dataset before and after removing empty columns/rows

In [ ]:
df_comparison = comparison_dfs(serie, serie_copy)
fig, ax1 = plt.subplots(figsize=(10, 10))
dt = df_comparison.melt(id_vars="Moment").rename(columns=str.title)
splot=sns.barplot(x="Moment", y="Value", data=dt, hue="Variable", ax=ax1)
for p in splot.patches:
    splot.annotate(format(p.get_height(), ".1f"), (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha="center", va="center", xytext=(0, 9), textcoords="offset points")
plt.xlabel("Comparison moments", size=12)
plt.ylabel("Number of records", size=12)
plt.title("Serie dataset before and after removing empty columns/rows", size=14)
sns.despine(fig)

### Comparing **`"Country-Serie"`** dataset before and after removing empty columns/rows

In [ ]:
df_comparison = comparison_dfs(country_serie, country_serie_copy)
fig, ax1 = plt.subplots(figsize=(10, 10))
dt = df_comparison.melt(id_vars="Moment").rename(columns=str.title)
splot=sns.barplot(x="Moment", y="Value", data=dt, hue="Variable", ax=ax1)
for p in splot.patches:
    splot.annotate(format(p.get_height(), ".1f"), (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha="center", va="center", xytext=(0, 9), textcoords="offset points")
plt.xlabel("Comparison moments", size=12)
plt.ylabel("Number of records", size=12)
plt.title("Country-Serie dataset before and after removing empty columns/rows", size=14)
sns.despine(fig)

### Comparing **`"FootNote"`** dataset before and after removing empty columns/rows

In [ ]:
df_comparison = comparison_dfs(foot_note, foot_note_copy)
fig, ax1 = plt.subplots(figsize=(10, 10))
dt = df_comparison.melt(id_vars="Moment").rename(columns=str.title)
splot=sns.barplot(x="Moment", y="Value", data=dt, hue="Variable", ax=ax1)
for p in splot.patches:
    splot.annotate(format(p.get_height(), ".1f"), (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha="center", va="center", xytext=(0, 9), textcoords="offset points")
plt.xlabel("Comparison moments", size=12)
plt.ylabel("Number of records", size=12)
plt.title("Foot Note dataset before and after removing empty columns/rows", size=14)
sns.despine(fig)

### Comparing **`"Data"`** dataset before and after removing empty columns/rows

In [ ]:
df_comparison = comparison_dfs(data, data_copy)
fig, ax1 = plt.subplots(figsize=(10, 10))
dt = df_comparison.melt(id_vars="Moment").rename(columns=str.title)
splot=sns.barplot(x="Moment", y="Value", data=dt, hue="Variable", ax=ax1)
for p in splot.patches:
    splot.annotate(format(p.get_height(), ".1f"), (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha="center", va="center", xytext=(0, 9), textcoords="offset points")
plt.xlabel("Comparison moments", size=12)
plt.ylabel("Number of records", size=12)
plt.title("Data dataset before and after removing empty columns/rows", size=14)
sns.despine(fig)

### Cleaning **`"Country"`** dataset